<a href="https://colab.research.google.com/github/tguerand/SentimentAnalysis/blob/master/bin/data_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import urllib.request
from sklearn.model_selection import train_test_split
print("Tensorflow version is :", tf.__version__)

Tensorflow version is : 2.3.0


#Access Path

In [2]:
!git clone https://github.com/tguerand/SentimentAnalysis
os.mkdir("./SentimentAnalysis/DATA")
#Download file
!gdown --id 1dO4aubOro159awBPzD675c22qtXMOVp0 --output ./SentimentAnalysis/DATA/training.csv

Cloning into 'SentimentAnalysis'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 59 (delta 17), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), done.
Downloading...
From: https://drive.google.com/uc?id=1dO4aubOro159awBPzD675c22qtXMOVp0
To: /content/SentimentAnalysis/DATA/training.csv
239MB [00:02, 87.9MB/s]


#Import Data

In [8]:
df=pd.read_csv("./SentimentAnalysis/DATA/training.csv",encoding='latin-1')
df.columns=['target', 't_id', 'created_at', 'query', 'user', 'text']
#df.head()

In [ ]:
print(df["text"].head(5).to_string())

0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 


#Data Analysis

#### Data preparation

In [16]:
#code Micka

df["nbhash"]=df["text"].str.count("#")>0
df["nbat"]=df["text"].str.count("@")>0
import re
df["text"] = df["text"].apply(lambda x : re.sub('@[a-zA-Z0-9]+', 'ààà', x))
df["text"] = df["text"].apply(lambda x : re.sub('http://[a-zA-Z0-9]+', 'urlofsite', x))
df["text"] = df["text"].apply(lambda x : re.sub('www.[a-zA-Z0-9]+', 'urlofsite', x))
df["text"] = df["text"].apply(lambda x : x.replace("!"," exclamationmark "))
df["text"] = df["text"].apply(lambda x : x.replace("?"," questionmark "))
df["text"] = df["text"].apply(lambda x : x.replace("&lt;3"," loveemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(":)"," smileemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(": )"," smileemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(":'("," cryemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(";)"," winkemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(":/"," mehemoji "))
df["text"] = df["text"].apply(lambda x : x.replace("w/o"," without "))
df["text"] = df["text"].apply(lambda x : x.replace("b/c"," because "))
df["text"] = df["text"].apply(lambda x : x.replace(" w "," with "))
df["text"] = df["text"].apply(lambda x : x.replace(" ... "," pointdesuspension "))
df["text"] = df["text"].apply(lambda x : x.replace(" .. "," pointdesuspension "))
df["text"] = df["text"].apply(lambda x : x.replace(" .... "," pointdesuspension "))


In [ ]:
#truc à garder ou à tester :
# les chiffres
# les ! et ?
# faire gaffe aux ', par exemple aren't ou didn't, qui expriment une négation
# http://twitpic
# http://bit
# http://tinyurl
# http://plurk
# http://www
# http://blip
# &lt;3 qui vaut <3
# :) ;) :/ mais aussi : ) ou :'(
# &amp &gt et &quot qui ont l'air d'être des infos html
# w/o without
# w/ with
# b/c because
# ... .. et .... 


In [ ]:
# Find which words are upper to remove ones whithout info ('I', ...)
#df["text"][0] = "I AAA"
irrelevant_words = ['I']
#df["redondant"] = df["text"].apply(lambda x: True if "I" in x.split() else False)
x = df["text"]
for word in irrelevant_words:
  x = x.str.replace(word,'')

# Check if there is a UPPERCASE word in the series
df["upper"] = x.str.split()
df["upper"] = df["upper"].apply(lambda x: [i.isupper() for i in x])
df["upper"] = df["upper"].apply(lambda x: True if True in x else False)
#df["upper"].head()
confusion_matrix = pd.crosstab(df["target"], df["upper"], rownames=['target'], colnames=['is any upper'])
confusion_matrix/df.shape[0]*100

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#------------------------Eliminate stop words-------------------------------
# Put everything to lower
df["text"]=df["text"].str.lower()
# remove stop_words
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
irrelevant_words = stopwords.words('english')
x = df["text"]
df["text"] = df["text"].apply(lambda x: [x.replace(word,'') for word in irrelevant_words])

In [ ]:
#------------------------Data words Frequency-------------------------------
treshhold=0.1
dict_frequency=dict(tokenizer.word_counts)
col=pd.Series(dict_frequency, name='freq')/22171669*100
df_freq=pd.DataFrame(col.sort_values(ascending=False))
df_freq.reset_index(drop=False)
df_freq=df_freq[df_freq["freq"]>treshhold]
df_freq['freq']

In [ ]:
df_copy=df
df_copy["length"]=df_copy["text"].str.len()
df["length"].value_counts()
#---------------------------Dataviz------------------------------------
import seaborn as sns
import numpy as np
sns.set_theme(); np.random.seed(0)
x = np.random.randn(100)
ax = sns.displot(df_copy, x="length", hue="target")

import seaborn as sns
sns.boxplot(
    data=df_copy,
    x='target',
    y='length',
    color='red')

#### With all data

In [10]:
vocab_size = 100
sentences = df['text']
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<00V>", filters='0123456789!"#$%&()*+,-./:;<=>?@[\\]^_{|}~\t\n')
tokenizer.fit_on_texts(sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(sentences)
padded=pad_sequences(sequences,padding="post")

In [15]:
word_index

{'<00V>': 1,
 'exclamationmark': 2,
 'ààà': 3,
 'i': 4,
 'to': 5,
 'the': 6,
 'a': 7,
 'my': 8,
 'and': 9,
 'you': 10,
 'questionmark': 11,
 'is': 12,
 'it': 13,
 'in': 14,
 'for': 15,
 'of': 16,
 'on': 17,
 'me': 18,
 'so': 19,
 'have': 20,
 'that': 21,
 'but': 22,
 "i'm": 23,
 'just': 24,
 'with': 25,
 'be': 26,
 'at': 27,
 'not': 28,
 'was': 29,
 'this': 30,
 'now': 31,
 'good': 32,
 'up': 33,
 'day': 34,
 'all': 35,
 'get': 36,
 'out': 37,
 'like': 38,
 'are': 39,
 'no': 40,
 'go': 41,
 'quot': 42,
 'urlofsite': 43,
 'do': 44,
 'today': 45,
 "it's": 46,
 'too': 47,
 'your': 48,
 'work': 49,
 'love': 50,
 'going': 51,
 'got': 52,
 'lol': 53,
 'time': 54,
 'am': 55,
 'back': 56,
 'from': 57,
 'u': 58,
 'one': 59,
 'what': 60,
 'com': 61,
 'will': 62,
 'im': 63,
 'know': 64,
 'we': 65,
 'about': 66,
 'really': 67,
 "don't": 68,
 'amp': 69,
 'had': 70,
 'can': 71,
 'see': 72,
 'some': 73,
 "can't": 74,
 'its': 75,
 'if': 76,
 'still': 77,
 'night': 78,
 'well': 79,
 'new': 80,
 'want':

#### Spliting data

In [ ]:
vocab_size = 100
sentences = df['text']
# Split Training and Testing
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df["target"], test_size=0.33, random_state=42)
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<00V>")
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index

train_sequences=tokenizer.texts_to_sequences(X_train)
train_padded=pad_sequences(train_sequences,padding="post")

test_sequences=tokenizer.texts_to_sequences(X_test)
test_padded=pad_sequences(test_sequences,padding="post")

#### Model and Training

In [ ]:
embedding_dim = 16
model=tf.keras.Sequential(
    [
     tf.keras.layers.Embedding(vocab_size,embedding_dim),
     tf.keras.layers.GlobalAveragePooling1D(),
     tf.keras.layers.Dense(24,activation="relu"),
     tf.keras.layers.Dense(1,activation="sigmoid")
    ]
    )
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit(train_padded,y_train,epochs=10,batch_size=100,validation_data=(test_padded,y_test))

Epoch 1/10
10720/10720 [==============================] - 33s 3ms/step - loss: -622879168.0000 - accuracy: 0.0000e+00 - val_loss: -708768064.0000 - val_accuracy: 0.0000e+00
Epoch 2/10
 7144/10720 [==================>...........] - ETA: 5s - loss: -731193984.0000 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored